# 基于转换的标注

N-Gram 标注器有两个潜在的问题：一是 n-gram 表的大小（语言模型)，使用回退标注器的 n-gram 标注器可能存储 bigram 和 trigram 表，这会是很大的稀疏矩阵，难以部署在移动计算设备上；二是关于上下文的，n-gram 标注器只使用上下文中的标记，而很难使用词的其它特征，虽然词本身可能是一个有用的信息源。

**Brill 标注**是一种基于转换的学习，它不断猜测每个词的标记，然后返回和修正错误，其转换规则是语言学可解释的。以下表为例，首先使用 unigram 标注器标注，然后运行两个规则：1.当前面词的词性是 TO 时，替换 NN 为 VB；2.当后一个标记是 NNS 时，替换 TO 为 IN。经过两条规则转换后得到最终的标注结果：

| Phrase | to | increase | grants | to | states | for | vocational | rehabilitation |
|:-------:|:--:|:--------:|:------:|:--:|:------:|:---:|:----------:|:--------------:|
| Unigram | TO | NN | NNS | TO | NNS | IN | JJ | NN |
| Rule 1 |   | VB |  |  |  |  |  |  |
| Rule 2 |   |   |   | IN |  |  |  |  |
| Output | TO | VB | NNS | IN | NNS | IN | JJ | NN |
| Gold | TO | VB | NNS | IN | NNS | IN | JJ | NN |

Brill 标注器的所有规则都是由以下形式的模板产生的：“替换 T1 为 T2 在上下文 C 中”。典型的上下文是之前或之后的词的内容或标记，或者当前词的两到三个词范围内出现一个特定的标记。在训练期间，T1，T2 和 C 的标注器猜测值创造出数以千计的候选规则，每一条规则根据其净收益打分：它修正的不正确标记的数目减去它错误修改的正确标记数目。

In [1]:
from nltk.tbl import demo
demo.demo()

Loading tagged data from treebank... 
Read testing data (200 sents/5251 wds)
Read training data (800 sents/19933 wds)
Read baseline data (800 sents/19933 wds) [reused the training set]
Trained baseline tagger
    Accuracy on test set: 0.8366
Training tbl tagger...
TBL train (fast) (seqs: 800; tokens: 19933; tpls: 24; min score: 3; min acc: None)
Finding initial useful rules...
    Found 12799 useful rules.

           B      |
   S   F   r   O  |        Score = Fixed - Broken
   c   i   o   t  |  R     Fixed = num tags changed incorrect -> correct
   o   x   k   h  |  u     Broken = num tags changed correct -> incorrect
   r   e   e   e  |  l     Other = num tags changed incorrect -> incorrect
   e   d   n   r  |  e
------------------+-------------------------------------------------------
  23  23   0   0  | POS->VBZ if Pos:PRP@[-2,-1]
  18  19   1   0  | NN->VB if Pos:-NONE-@[-2] & Pos:TO@[-1]
  14  14   0   0  | VBP->VB if Pos:MD@[-2,-1]
  12  12   0   0  | VBP->VB if Pos:TO@[-1]
  